# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,7.68,93,90,0.45,NZ,1699465669
1,1,tofol,5.3248,163.0078,28.97,77,85,6.58,FM,1699465669
2,2,hamilton,39.1834,-84.5333,25.55,59,20,7.72,US,1699465572
3,3,somanda,-3.3667,33.9500,22.69,71,81,5.93,TZ,1699465669
4,4,tralee,52.2704,-9.7026,7.14,87,40,5.66,IE,1699465669


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    scale=0.5,
    color="City",
    frame_width = 800,
    frame_height = 600,
    alpha=0.7
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness
max_temp = city_data_df["Max Temp"] < 27
min_temp = city_data_df["Max Temp"] > 21
wind_speed = city_data_df["Wind Speed"] < 4.5
zero_clouds = city_data_df["Cloudiness"] == 0

ideal_weather_df = city_data_df[max_temp & min_temp & wind_speed & zero_clouds]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,63,hawaiian paradise park,19.5933,-154.9731,22.80,80,0,3.60,US,1699465534
127,127,tamanrasset,22.7850,5.5228,23.95,9,0,2.06,DZ,1699465692
239,239,chillicothe,39.3331,-82.9824,22.18,52,0,4.12,US,1699465445
300,300,as sulayyil,20.4607,45.5779,26.14,25,0,4.04,SA,1699465726
428,428,tubarjal,30.4999,38.2160,24.05,24,0,2.81,SA,1699465745
486,486,guarapari,-20.6667,-40.4975,26.05,50,0,2.90,BR,1699465624
548,548,bani mazar,28.5036,30.8004,25.81,38,0,3.96,EG,1699465775


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
63,hawaiian paradise park,US,19.5933,-154.9731,80,
127,tamanrasset,DZ,22.7850,5.5228,9,
239,chillicothe,US,39.3331,-82.9824,52,
300,as sulayyil,SA,20.4607,45.5779,25,
428,tubarjal,SA,30.4999,38.2160,24,
486,guarapari,BR,-20.6667,-40.4975,50,
548,bani mazar,EG,28.5036,30.8004,38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
categories = "accommodation.hotel"

params = {
    "limit":limit,
    "categories":categories,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, 'Lat']
    longitude = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
tamanrasset - nearest hotel: فندق الأمان
chillicothe - nearest hotel: Quality Inn Chillicothe
as sulayyil - nearest hotel: No hotel found
tubarjal - nearest hotel: No hotel found
guarapari - nearest hotel: Osmar Hotel
bani mazar - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
63,hawaiian paradise park,US,19.5933,-154.9731,80,No hotel found
127,tamanrasset,DZ,22.7850,5.5228,9,فندق الأمان
239,chillicothe,US,39.3331,-82.9824,52,Quality Inn Chillicothe
300,as sulayyil,SA,20.4607,45.5779,25,No hotel found
428,tubarjal,SA,30.4999,38.2160,24,No hotel found
486,guarapari,BR,-20.6667,-40.4975,50,Osmar Hotel
548,bani mazar,EG,28.5036,30.8004,38,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    frame_width = 800,
    frame_height = 600,
    hover_cols=['Hotel Name','Country']
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)